# LangGraph 

- Node, Edge, State를 통해 LLM을 활용한 워크플로우에 순환(Cycle) 연산 기능을 추가하여 손쉽게 흐름을 제어. 
- RAG 파이프라인의 세부 단계별 흐름제어가 가능 
- Conditional Edge: 조건부 (if, elif, else와 같은) 흐름 제어 
- Human-in-the-loop: 필요시 중간 개입하여 다음 단계를 결정 
- Checkpointer: 과거 실행 과정에 대한 "수정" & "리플레이" 기능 


# 목차 
- 1. 상태 (State)
- 2. 노드 (Node)
- 3. 엣지 (Edge)
- 4. 조건부 엣지 (Conditional Edge)
- 5. 시작점 지정 
- 6. 체크포인터 (memory)
- 7. 그래프 시각화
- 8. 그래프 실행

# 1. 상태 (State)

노드와 노드간에 정보를 전달할 때 상태(State) 객체에 담아 전달 

- TypedDict: 일반 파이썬 dict에 타입힌트를 추가한 개념, dictionary와 유사 
- **모든 값을 다 채우지 않아도 된다**
- 새로운 노드에서 값을 덮어쓰기 방식으로 채운다 
- Reducer (add_message 혹은 operator.add)
    + 자동으로 list에 기존 값을 유지하면서 새로운 메시지를 추가하는 기능 


In [1]:
from typing import TypedDict, Annotated, List
from langgraph.graph.message import add_messages
from langchain_core.documents import Document
import operator

# State 정의
class GraphState(TypedDict):
    question: Annotated[list, add_messages]
    context: Annotated[str, "Context"]
    answer: Annotated[str, "Answer"]
    message: Annotated[list, add_messages]
    relevance: Annotated[str, "Relevance"]
    
    
# # State 정의
# class GraphState(TypedDict):
#     question: Annotated[str, "user question"]
#     context: Annotated[List[Document], operator.add]
#     answer: Annotated[List[Document], operator.add]
#     sql_query: Annotated[str, "sql query"]
#     binary_score: Annotated[str, "binary score yes or no"]

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph.message import add_messages

msgs1 = [HumanMessage(content="안녕하세요?", id="1")]
msgs2 = [AIMessage(content="반갑습니다~", id="2")]

# msgs1에 msgs2를 넣어줌 
result1 = add_messages(msgs1, msgs2)

In [5]:
result1

[HumanMessage(content='안녕하세요?', additional_kwargs={}, response_metadata={}, id='1'),
 AIMessage(content='반갑습니다~', additional_kwargs={}, response_metadata={}, id='2')]

# 2. 노드 (Node)

- 함수로 정의 
- 입력인자: 상태(State) 객체 
- 반환 (return)
    - 대부분 상태(State) 객체 
    - Conditional Edge의 경우 다를 수 있음 

코드

```python
def retrieve_document(state: GraphState) -> GraphState:
    # 문서에서 검색하여 관련성 있는 문서를 찾습니다.
    retrieved_docs = pdf_retriever.invoke(state["question"])
    # 검색된 문서를 context 키에 저장합니다.
    return GraphState(context=format_docs(retrieved_docs))
```

## Graph 생성 후 노드 추가 

- 이전에 정의한 함수를 Graph에 추가 
- add_node("노드이름", 함수)

코드

```python

from typing import TypedDict, Annotated
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages

# State 정의
class GraphState(TypedDict):
    question: Annotated[list, add_messages]
    context: Annotated[str, "Context"]
    answer: Annotated[str, "Answer"]
    message: Annotated[list, add_messages]
    relevance: Annotated[str, "Relevance"]

# 노드 1 
# 문서에서 관련성 있는 문서를 찾는 노드 
def retrieve_document(state: GraphState) -> GraphState:
    # Question에 대한 문서 검색을 retriever로 수행 
    retrieved_docs = pdf_retriever.invoke(state["question"])
    # 검색된 문서를 context 키에 저장
    return GraphState(context=format_docs(retrieved_docs))

# 노드 2
# Chain을 사용하여 답변을 생성 
def llm_answer(state: GraphState) -> GraphState:
    return GraphState(answer=pdf_chain.invoke({"question": state["question"], "context": state["context"]}))


# langgraph.graph에서 StateGraph와 END를 가져온다.
workflow = StateGraph(GraphState)

# 정의된 함수를 Graph에 node로 넣어준다.
workflow.add_node("retrieve", retrieve_document)    # 정보 검색 노드에 이름을 지어주어 Graph에 추가 
workflow.add_node("llm_answer", llm_answer)         # 답변 생성기 노드에 이름을 지어주어 Graph에 추가 

```

# 3. 엣지 (Edge)

- 노드에서 노드간의 연결 
- add_edge("노드이름", "노드이름)
    - add_edge(from, to)

코드

```python
workflow.add_edge("retrieve", "llm_answer")  # 검색 -> 답변 
workflow.add_edge("llm_answer", "relevance_check")  # 답변 -> 관련성 체크 
```

# 4. 조건부 엣지 (Conditional Edge)

- 노드에 조건부 엣지를 추가하여 분기를 나눌 수 있음 
- add_conditional_edge("노드이름", 조건부판단함수, dict로 다음 단계 결정)

## 흐름 

- "relevance_check" 노드에서 나온 결과를 is_relevant 함수에 입력 
- 반환된 값은 "grounded", "notGrounded", "notSure" 중 하나 
    - value에 해당하는 값이 END면 Graph 실행 종료 
    - "llm_answer"와 같이 노드이름이면 해당 노드로 연결 

코드

```python
# 조건부 엣지를 Graph에 추가 
workflow.add_conditional_edges(
    "relevance_check",                  # "relevance_check"(관련성 체크)노드에서 나온 결과를 
    is_relevant,                        # is_relevant 함수에 전달 
    {
        "grounded": END,                 # 관련성이 (grounded)있으면 종료(END) 
        "notGrounded": "llm_answer",     # 관련성이 (notGrounded)없으면 다시 답변을 생성 (llm_answer 노드 실행)
        "notSure": "llm_answer",         # 관련성 체크 결과가 (notSure)모호하면 다시 답변을 생성 (llm_answer 노드 실행)
    },
)
```

# 5. 시작점 지정 

- set_entry_point("노드이름")
- 지정한 시작점부터 Graph가 시작 


코드

```python
# 시작점을 retreive 노드로 시작
workflow.set_entry_point("retrieve")
```


# 6. 체크포인터 (memory)

- Checkpointer: **각 노드간 실행결과를 추적하기 위한 메모리** (대화에 대한 기록과 유사 개념)
- 체크포인터를 활용하여 **특정 시점 (Snapshot)으로 되돌리기 기능** 가능 
- compile(checkpointer=memory) 지정하여 그래프 생성 

코드

```python
# 기록을 위한 메모리 저장소 설정 
memory = MemorySaver()
# 그래프를 컴파일 
app = workflow.compile(checkpointer=memory)
```

# 7. 그래프 시각화 

- get_graph(xray=True).draw_mermaid_png()
    - 생성한 그래프 시각화 


코드

```python
display(
    Image(app.get_graph(xray=True).draw_mermaid_png())
)

```


# 8. 그래프 실행 

- RunnableConfig 
    - recursion_limit: 최대 노드 실행 개수를 지정 
        + 무한 순환 구조에 빠지는 것을 방지
    - thread_id: 그래프 실행 아이디를 기록하고, 추후 추적하기 위한 목적으로 활용 

- 상태 (State)로 시작 
    - "question"에 질문만 입력하고 상태를 첫 번째 노드에게 전달 

- invoke(상태, config) 전달하여 실행  

코드 

```python

from langchain_core.runnables import RunnableConfig

# recursion_limit: 최대 반복 횟수, thread_id: 실행 ID (구분용)
config = RunnableConfig(recursion_limit=20, configurable={'thread_id': "test-graph"})

# GraphState 객체를 활용 
inputs = GraphState(question="삼성전자가 개발한 생성형 AI의 이름은?")
output = app.invoke(inputs, config=config)

```

## 결과 확인 
- 출력된 결과로 최종 확인 
- 출력 결과 역시 상태 (State)에 담겨 있음 

```python
# 출력 결과를 확인 
print("Question: \t", output["question"])
print("Answer: \t", output["question"])
print("Relevance: \t", output["relevance"])
```
